In [1]:
import numpy as np
import pandas as pd
from copy import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import SimpleImputer 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, hamming_loss
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import wandb

In [2]:
data = pd.read_csv('WineQT.csv', header=0)

In [3]:
train_data, temp_data = train_test_split(data, test_size=0.4,random_state = 42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state = 42)

In [4]:
scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Fit and transform on the training data
imputer = SimpleImputer(strategy='mean')  # You can choose 'mean', 'median', or other strategies

# Fit and transform on the training data
train_data_scaled = scaler.fit_transform(imputer.fit_transform(train_data.drop(columns=['quality'])))
train_labels = train_data['quality']

# Transform the validation and test data using the same scalers and imputer
validation_data_scaled = scaler.transform(imputer.transform(validation_data.drop(columns=['quality'])))
test_data_scaled = scaler.transform(imputer.transform(test_data.drop(columns=['quality'])))

# If you also want to normalize the data, you can do it using the MinMaxScaler
train_data_scaled_normalized = minmax_scaler.fit_transform(train_data_scaled)
validation_data_scaled_normalized = minmax_scaler.transform(validation_data_scaled)
test_data_scaled_normalized = minmax_scaler.transform(test_data_scaled)

# print(train_labels)
# print(train_data_scaled_normalized)
# print(test_data_scaled_normalized.shape)
# print(validation_data_scaled_normalized.shape)
X_train = train_data_scaled
y_train = train_labels
X_val = validation_data_scaled
y_val = validation_data['quality']
X_test = test_data_scaled
y_test = validation_data['quality']
y_train = np.array(y_train)
# y_val = np.array(y_val)
y_val = pd.get_dummies(y_val).astype(int)
y_val = np.array(y_val)
y_train = pd.get_dummies(y_train).astype(int)
y_train = np.array(y_train)
y_test = pd.get_dummies(y_test).astype(int)
y_test = np.array(y_test)
print(X_train)
print(y_train)


[[-1.67058247  1.20622441 -0.91067852 ... -0.59135112  1.0674252
   1.64361099]
 [-0.548498    0.69262724 -1.37982978 ... -1.11462937 -0.42042679
  -1.73858966]
 [-0.0169843   1.14915806 -0.28514351 ... -0.82391923 -0.88538054
  -0.31652802]
 ...
 [-0.48944092 -1.01936335  0.3403915  ... -0.358783    1.34639745
   1.62012349]
 [-0.19415553  0.23609641  0.23613566 ...  0.16449526 -0.88538054
   0.84290061]
 [-0.43038384 -0.04923535  0.70528692 ... -0.12621489  1.2534067
   1.60944735]]
[[0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 1 0 0 0]
 ...
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]]


In [5]:
# class Neural_Network(object):
#     def __init__(self, learning_rate=0.01,epochs = 5000, inputLayerSize: int = 2, hiddenLayerSize: int = 3, outputLayerSize: int = 1):
#         # Define Hyperparameters
#         self.inputLayerSize = inputLayerSize
#         self.outputLayerSize = outputLayerSize
#         self.hiddenLayerSize = hiddenLayerSize
#         self.activation = self.relu
#         self.activationPrime = self.reluPrime
#         self.learning_rate = learning_rate
#         # Weights (parameters)
#         self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
#         self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
#         self.optimizer = 'SGD'
#         self.max_iterations = epochs

#     def forward(self, X):
#         self.z2 = np.dot(X, self.W1)
#         self.a2 = self.activation(self.z2)  # Use ReLU activation function
#         self.z3 = np.dot(self.a2, self.W2)
#         yHat = self.softmax(self.z3)
#         return yHat

#     def relu(self, z):
#         return np.maximum(0, z)  # ReLU activation

#     def tanh(self, z):
#         return np.tanh(z)  # Tanh activation

#     def sigmoid(self, z):
#         return 1 / (1 + np.exp(-z))

#     def sigmoidPrime(self, z):
#         return np.exp(-z) / ((1 + np.exp(-z))**2)

#     def softmax(self, z):
#         exp_z = np.exp(z - np.max(z))  # Subtracting max(z) for numerical stability
#         return exp_z / exp_z.sum(axis=1, keepdims=True)

#     def costFunction(self, X, y):
#         self.yHat = self.forward(X)
#         J = -np.sum(y * np.log(self.yHat)) / len(X)
#         return J

#     def costFunctionPrime(self, X, y):
#         self.yHat = self.forward(X)

#         delta3 = self.yHat - y
#         dJdW2 = np.dot(self.a2.T, delta3)

#         delta2 = np.dot(delta3, self.W2.T) * self.activationPrime(self.z2)  # Use ReLU derivative
#         dJdW1 = np.dot(X.T, delta2)

#         return dJdW1, dJdW2

#     def reluPrime(self, z):
#         return np.where(z > 0, 1, 0)  # Derivative of ReLU

#     def tanhPrime(self, z):
#         return 1 - np.tanh(z)**2  # Derivative of Tanh

#     def backward(self, X, y, learning_rate):
#         dJdW1, dJdW2 = self.costFunctionPrime(X, y)
#         self.W1 -= learning_rate * dJdW1
#         self.W2 -= learning_rate * dJdW2
#     def train(self, X, y, batch_size = 32, learning_rate=0.1, max_iterations=10000):
#         if(self.optimizer == 'SGD'):
#             return self.train_sgd(X=X, y=y)
#         elif(self.optimizer == 'Batch'):
#             return self.train_batch(X=X, y=y)
#         elif(self.optimizer == 'MiniBatch'):
#             return self.train_mini_batch(X=X, y=y)
#     def train_sgd(self, X, y, learning_rate=0.1, max_iterations=10000):
#         for i in range(self.max_iterations):
#             idx = np.random.randint(len(X))
#             self.backward(X[idx:idx+1], y[idx:idx+1], learning_rate)
#             # if i % 1000 == 0:
#             #     loss = self.costFunction(X, y)
#             #     print("Iteration %d: loss = %f" % (i, loss))
#         loss = self.costFunction(X, y)
#         return loss
#         print("Training completed.")
#     def train_batch(self, X, y, learning_rate=0.0001, max_iterations=10000):
#         for i in range(self.max_iterations):
#             self.backward(X, y, learning_rate)
#             # if i % 1000 == 0:
#             #     loss = self.costFunction(X, y)
#             #     print("Iteration %d: loss = %f" % (i, loss))
#         loss = self.costFunction(X, y)
#         return loss
#         print("Training completed.")
#     def train_mini_batch(self, X, y, batch_size=32, learning_rate=0.01, max_iterations=5000):
#         for i in range(self.max_iterations):
#             # Randomly shuffle the data and split into mini-batches
#             indices = np.arange(len(X))
#             np.random.shuffle(indices)
#             for j in range(0, len(X), batch_size):
#                 batch_indices = indices[j:j+batch_size]
#                 self.backward(X[batch_indices], y[batch_indices], learning_rate)
#             if i % 1 == 0:
#                 loss = self.costFunction(X, y)
#                 print("Iteration %d: loss = %f" % (i, loss))
#         loss = self.costFunction(X, y)
#         return loss
#         print("Training completed.")

#     def predict(self, X):
#         yHat = self.forward(X)
#         # return yHat
#         binary_predictions = np.zeros_like(yHat)
#         binary_predictions[np.arange(len(yHat)), yHat.argmax(axis=1)] = 1
#         return binary_predictions
#         # return np.argmax(yHat, axis=1)
#     def set_learning_rate(self, learning_rate):
#         self.learning_rate = learning_rate

#     def set_activation_function(self, activation_function):
#         if(activation_function == 'sigmoid'):
#             self.activation = self.sigmoid
#             self.activationPrime = self.sigmoidPrime
#         if(activation_function == 'relu'):
#             self.activation = self.relu
#             self.activationPrime = self.reluPrime
#         if(activation_function == 'tanh'):
#             self.activation = self.tanh
#             self.activationPrime = self.tanhPrime

#     def set_optimizer(self, optimizer):
#         self.optimizer = optimizer

#     def set_hidden_layers(self, hidden_layer_sizes):
#         self.hidden_layer_sizes = hidden_layer_sizes
#         self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
#         self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
#         # self.weights, self.biases = self.initialize_weights()
#     def set_epochs(self, epcohs):
#         self.max_iterations = epochs
#         print(self.max_iterations)


In [6]:
class Neural_Network(object):
    def __init__(self, learning_rate=0.01,epochs = 5000, inputLayerSize: int = 2, hiddenLayerSize = [3,3,3] , outputLayerSize: int = 1):
        # Define Hyperparameters
        self.inputLayerSize = inputLayerSize
        self.outputLayerSize = outputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        self.activation = self.relu
        self.activationPrime = self.reluPrime
        self.learning_rate = learning_rate
        # Weights (parameters)
        self.num_layers = len(hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        self.optimizer = 'SGD'
        self.max_iterations = epochs

    def forward(self, X):
        self.z = [None] * (len(self.weights))
        self.a = [None] * (len(self.weights)-1)
        
        self.z[0] = np.dot(X, self.weights[0])
        for i in range(0,len(self.a)):
            self.a[i] = self.activation(self.z[i])
            self.z[i+1] = np.dot(self.a[i],self.weights[i+1])
        yHat = self.softmax(self.z[-1])
        return yHat

    def relu(self, z):
        return np.maximum(0, z)  # ReLU activation

    def tanh(self, z):
        return np.tanh(z)  # Tanh activation

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoidPrime(self, z):
        return np.exp(-z) / ((1 + np.exp(-z))**2)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z))  # Subtracting max(z) for numerical stability
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        J = -np.sum(y * np.log(self.yHat)) / len(X)
        return J

    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        deltas = [None] * (len(self.weights))
        deltas[-1] = self.yHat - y
        dJdWs = [None] * (len(self.weights))
        for i in range(len(deltas)-2,-1,-1):
            dJdWs[i+1] = np.dot(self.a[i].T, deltas[i+1])
            deltas[i] = np.dot(deltas[i+1],self.weights[i+1].T)*self.activationPrime(self.z[i])
        dJdWs[0] = np.dot(X.T, deltas[0])
        
        # delta3 = self.yHat - y
        # dJdW2 = np.dot(self.a2.T, delta3)

        # delta2 = np.dot(delta3, self.W2.T) * self.activationPrime(self.z2)  # Use ReLU derivative
        # dJdW1 = np.dot(X.T, delta2)

        return dJdWs

    def reluPrime(self, z):
        return np.where(z > 0, 1, 0)  # Derivative of ReLU

    def tanhPrime(self, z):
        return 1 - np.tanh(z)**2  # Derivative of Tanh

    def backward(self, X, y, learning_rate):
        dJdWs = self.costFunctionPrime(X, y)
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * dJdWs[i]
        # self.W1 -= learning_rate * dJdW1
        # self.W2 -= learning_rate * dJdW2
    def train(self, X, y, batch_size = 32, learning_rate=0.1, max_iterations=10000,wand = 0):
        if(self.optimizer == 'SGD'):
            return self.train_sgd(X=X, y=y,wand=wand)
        elif(self.optimizer == 'Batch'):
            return self.train_batch(X=X, y=y,wand=wand)
        elif(self.optimizer == 'MiniBatch'):
            return self.train_mini_batch(X=X, y=y,wand=wand)
    def train_sgd(self, X, y, learning_rate=0.1, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on WineQT MLP")
        for i in range(self.max_iterations):
            idx = np.random.randint(len(X))
            self.backward(X[idx:idx+1], y[idx:idx+1], self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_batch(self, X, y, learning_rate=0.0001, max_iterations=10000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on WineQT MLP")
        for i in range(self.max_iterations):
            self.backward(X, y, self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_mini_batch(self, X, y, batch_size=32, learning_rate=0.01, max_iterations=5000, wand = 0):
        if(wand == 1):
            wandb.init(project="Reporting loss on WineQT MLP")
        for i in range(self.max_iterations):
            # Randomly shuffle the data and split into mini-batches
            indices = np.arange(len(X))
            np.random.shuffle(indices)
            for j in range(0, len(X), batch_size):
                batch_indices = indices[j:j+batch_size]
                self.backward(X[batch_indices], y[batch_indices], self.learning_rate)
            if i % 1 == 0:
                loss = self.costFunction(X, y)
                # print("Iteration %d: loss = %f" % (i, loss))
                if(wand == 1):
                    wandb.log({"Loss": loss, "Epoch": i})
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")

    def predict(self, X):
        yHat = self.forward(X)
        # return yHat
        binary_predictions = np.zeros_like(yHat)
        binary_predictions[np.arange(len(yHat)), yHat.argmax(axis=1)] = 1
        return binary_predictions
        # return np.argmax(yHat, axis=1)
    def set_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def set_activation_function(self, activation_function):
        if(activation_function == 'sigmoid'):
            self.activation = self.sigmoid
            self.activationPrime = self.sigmoidPrime
        if(activation_function == 'relu'):
            self.activation = self.relu
            self.activationPrime = self.reluPrime
        if(activation_function == 'tanh'):
            self.activation = self.tanh
            self.activationPrime = self.tanhPrime

    def set_optimizer(self, optimizer):
        self.optimizer = optimizer

    def set_hidden_layers(self, hidden_layer_sizes):
        self.hiddenLayerSize = hidden_layer_sizes
        self.num_layers = len(self.hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        # self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
        # self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
        # self.weights, self.biases = self.initialize_weights()
    def set_epochs(self, epcohs):
        self.max_iterations = epochs
        print(self.max_iterations)


In [7]:
wandb.login()

wandb: Currently logged in as: lsamourya07 (mourya). Use `wandb login --relogin` to force relogin


True

In [8]:
np.random.seed(27)
mlp_sgd = Neural_Network(learning_rate=0.01,epochs=5000, inputLayerSize=12, hiddenLayerSize=[18], outputLayerSize=6)
mlp_sgd.set_activation_function('sigmoid')
mlp_sgd.set_optimizer('SGD')
loss = mlp_sgd.train(X_train, y_train,wand=1)
wandb.init(project="MLP Classification on WineQT.csv")
y_pred = mlp_sgd.predict(X_val)
acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro') 
precision = precision_score(y_val, y_pred, average='macro')
recall = recall_score(y_val, y_pred, average='macro')
print("Loss:", loss)
print("Accuracy:", acc)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
y_train_pred = mlp_sgd.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred, average='macro')  
train_precision = precision_score(y_train, y_train_pred, average='macro')
train_recall = recall_score(y_train, y_train_pred, average='macro')
print("Loss:", loss)
print("Accuracy:", train_accuracy)
print("F1 Score:", train_f1)
print("Precision:", train_precision)
print("Recall:", train_recall)
print(acc)
print(train_accuracy)
wandb.log({"Loss": loss, "Accuracy": acc})
wandb.log({"Loss": loss, "Accuracy": train_accuracy})

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▅▃▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,4999
Loss,0.93207


Loss: 0.9320701134595549
Accuracy: 0.62882096069869
F1 Score: 0.28086370197375893
Precision: 0.31639826991828396
Recall: 0.28161110665152955
Loss: 0.9320701134595549
Accuracy: 0.6116788321167883
F1 Score: 0.2959359197423386
Precision: 0.3792398709252854
Recall: 0.2880217153732829
0.62882096069869
0.6116788321167883


/home/ubuntulsa/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntulsa/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

# Define the sweep configuration
sweep_config = {
    "name": "example_sweep",
    "method": "grid",  # Other options: "grid", "bayes"
    "metric": {"goal": "maximize", "name": "accuracy"},
    "parameters": {
        "activation": {"values": ["sigmoid", "tanh", "relu"]},
        "optimizer": {"values": ["SGD", "Batch", "MiniBatch"]},
    },
    "early_terminate": {
        "type": "hyperband",
        "s": 2,
        "eta": 3,
        "max_iter": 27,
    },
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="your_project_name")

# Define your training function
def train():
    # Access sweep configuration
    wandb.init(project="your_project_name")
    
    config = wandb.config

    # Your training code here, using hyperparameters from config
    # Example:
    activation = config.activation
    optimizer = config.optimizer

    # Log metrics
    np.random.seed(27)
    mlp_sgd = Neural_Network(learning_rate=0.01,epochs=5000, inputLayerSize=12, hiddenLayerSize=[18], outputLayerSize=6)
    mlp_sgd.set_activation_function(activation)
    mlp_sgd.set_optimizer(optimizer)
    loss = mlp_sgd.train(X_train, y_train)
    y_pred = mlp_sgd.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro') 
    precision = precision_score(y_val, y_pred, average='macro')
    recall = recall_score(y_val, y_pred, average='macro')
    wandb.log({"activation": activation, "optimizer": optimizer, "accuracy": acc})

# Start the sweep
wandb.agent(sweep_id, function=train)


In [9]:
# Initialize a wandb sweep.
sweep_config = {
    'method': 'grid',
    'name': 'grid_search_example',
    'parameters': {
        'activation': {
            'values': ["sigmoid", "tanh", "relu"]
        },
        'optimizer': {
            'values': ["SGD", "Batch", "MiniBatch"]
        },
        'learning_rate': {
            'values': [0.01, 0.001]
        },
        'epochs': {
            'values': [100, 50]
        },
        'hiddenlayersize': {
            'values': [
                [14],
                [18],
                [18, 18]
            ]
        }
    }
}

# Initialize your sweep.
sweep_id = wandb.sweep(sweep_config, project="Hyperparameter tuning on Multiclass classification")

# This function will be called for each combination of hyperparameters.
def train_neural_network():
    wandb.init(project="Hyperparameter tuning on Multiclass classification")
    # Inside this function, you should set your model and train it using the current hyperparameters.
    # Replace this part with your model training logic.
    mlp_sgd = Neural_Network(learning_rate=wandb.config.learning_rate, epochs=wandb.config.epochs, inputLayerSize=12, hiddenLayerSize=wandb.config.hiddenlayersize, outputLayerSize=6)
    mlp_sgd.set_activation_function(wandb.config.activation)
    mlp_sgd.set_optimizer(wandb.config.optimizer)
    loss = mlp_sgd.train(X_train, y_train)
    y_pred = mlp_sgd.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro') 
    precision = precision_score(y_val, y_pred, average='macro')
    recall = recall_score(y_val, y_pred, average='macro')
    wandb.log({"Accuracy": acc, "F1 Score": f1, "Precision": precision, "Recall": recall})

# Run the sweep.
wandb.agent(sweep_id, function=train_neural_network)

# You can access the results using the wandb API after the sweep is complete.


Create sweep with ID: 8vlz4wtj
Sweep URL: https://wandb.ai/mourya/Hyperparameter%20tuning%20on%20Multiclass%20classification/sweeps/8vlz4wtj


wandb: Agent Starting Run: 7eredkww with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 100
wandb: 	hiddenlayersize: [14]
wandb: 	learning_rate: 0.01
wandb: 	optimizer: SGD
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/home/ubuntulsa/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.47598
F1 Score,0.1706
Precision,0.16663
Recall,0.19275


wandb: Agent Starting Run: nvhob7mf with config:
wandb: 	activation: sigmoid
wandb: 	epochs: 100
wandb: 	hiddenlayersize: [14]
wandb: 	learning_rate: 0.01
wandb: 	optimizer: Batch
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

/home/ubuntulsa/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.53712
F1 Score,0.25289
Precision,0.28953
Recall,0.23936


wandb: Agent Starting Run: nru541nh with config:
wandb: 	activation: relu
wandb: 	epochs: 50
wandb: 	hiddenlayersize: [18, 18]
wandb: 	learning_rate: 0.001
wandb: 	optimizer: MiniBatch
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.62882
F1 Score,0.35598
Precision,0.37284
Recall,0.35322


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
np.random.seed(41)
mlp_sgd = Neural_Network(learning_rate=0.01,epochs=5000, inputLayerSize=12, hiddenLayerSize=[18], outputLayerSize=6)
mlp_sgd.set_activation_function('relu')
mlp_sgd.set_optimizer('Batch')
mlp_sgd.set_learning_rate(0.01)
# print(l)
mlp_sgd.max_iterations = 500;
mlp_sgd.set_hidden_layers([18])
loss = mlp_sgd.train(X_train, y_train)
y_pred = mlp_sgd.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro') 
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
print(acc, loss)

###### 

In [ ]:
class MultilabelMLP(object):
    def __init__(self, learning_rate=0.01,epochs = 5000, inputLayerSize: int = 2, hiddenLayerSize = [3,3,3] , outputLayerSize: int = 1):
        # Define Hyperparameters
        self.inputLayerSize = inputLayerSize
        self.outputLayerSize = outputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        self.activation = self.relu
        self.activationPrime = self.reluPrime
        self.learning_rate = learning_rate
        # Weights (parameters)
        self.num_layers = len(hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        self.optimizer = 'SGD'
        self.max_iterations = epochs

    def forward(self, X):
        self.z = [None] * (len(self.weights))
        self.a = [None] * (len(self.weights)-1)
        
        self.z[0] = np.dot(X, self.weights[0])
        for i in range(0,len(self.a)):
            self.a[i] = self.activation(self.z[i])
            self.z[i+1] = np.dot(self.a[i],self.weights[i+1])
        yHat = self.softmax(self.z[-1])
        return yHat

    def relu(self, z):
        return np.maximum(0, z)  # ReLU activation

    def tanh(self, z):
        return np.tanh(z)  # Tanh activation

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoidPrime(self, z):
        return np.exp(-z) / ((1 + np.exp(-z))**2)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z))  # Subtracting max(z) for numerical stability
        return exp_z / exp_z.sum(axis=1, keepdims=True)

    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        J = -np.sum(y * np.log(self.yHat)) / len(X)
        return J

    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        deltas = [None] * (len(self.weights))
        deltas[-1] = self.yHat - y
        dJdWs = [None] * (len(self.weights))
        for i in range(len(deltas)-2,-1,-1):
            dJdWs[i+1] = np.dot(self.a[i].T, deltas[i+1])
            deltas[i] = np.dot(deltas[i+1],self.weights[i+1].T)*self.activationPrime(self.z[i])
        dJdWs[0] = np.dot(X.T, deltas[0])
        
        # delta3 = self.yHat - y
        # dJdW2 = np.dot(self.a2.T, delta3)

        # delta2 = np.dot(delta3, self.W2.T) * self.activationPrime(self.z2)  # Use ReLU derivative
        # dJdW1 = np.dot(X.T, delta2)

        return dJdWs

    def reluPrime(self, z):
        return np.where(z > 0, 1, 0)  # Derivative of ReLU

    def tanhPrime(self, z):
        return 1 - np.tanh(z)**2  # Derivative of Tanh

    def backward(self, X, y, learning_rate):
        dJdWs = self.costFunctionPrime(X, y)
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * dJdWs[i]
        # self.W1 -= learning_rate * dJdW1
        # self.W2 -= learning_rate * dJdW2
    def train(self, X, y, batch_size = 32, learning_rate=0.1, max_iterations=10000):
        if(self.optimizer == 'SGD'):
            return self.train_sgd(X=X, y=y)
        elif(self.optimizer == 'Batch'):
            return self.train_batch(X=X, y=y)
        elif(self.optimizer == 'MiniBatch'):
            return self.train_mini_batch(X=X, y=y)
    def train_sgd(self, X, y, learning_rate=0.1, max_iterations=1000):
        for i in range(self.max_iterations):
            idx = np.random.randint(len(X))
            self.backward(X[idx:idx+1], y[idx:idx+1], learning_rate)
            # if i % 1000 == 0:
            #     loss = self.costFunction(X, y)
            #     print("Iteration %d: loss = %f" % (i, loss))
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_batch(self, X, y, learning_rate=0.0001, max_iterations=1000):
        for i in range(self.max_iterations):
            self.backward(X, y, learning_rate)
            # if i % 1000 == 0:
            #     loss = self.costFunction(X, y)
            #     print("Iteration %d: loss = %f" % (i, loss))
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")
    def train_mini_batch(self, X, y, batch_size=32, learning_rate=0.01, max_iterations=1000):
        for i in range(self.max_iterations):
            # Randomly shuffle the data and split into mini-batches
            indices = np.arange(len(X))
            np.random.shuffle(indices)
            for j in range(0, len(X), batch_size):
                batch_indices = indices[j:j+batch_size]
                self.backward(X[batch_indices], y[batch_indices], learning_rate)
            # if i % 1 == 0:
            #     loss = self.costFunction(X, y)
            #     print("Iteration %d: loss = %f" % (i, loss))
        loss = self.costFunction(X, y)
        return loss
        print("Training completed.")

    def predict(self, X, y):
        yHat = self.forward(X)
        # print(X)
        # print(y)
        # print(yHat)
        # binary_predictions = []
        # for i in range(len(y)):
        #     count = np.sum(y[i] > 0)
        #     print(count)
        #     yHat[i] = yHat[i]*count
        #     temp = (yHat[i] > 0.5).astype(int)
        #     lis = temp/np.sum(temp)
        #     lis = lis.tolist()
        #     binary_predictions.append(lis)
        # return np.array(binary_predictions)
        binary_predictions = (yHat > 0.2).astype(int)
        return np.array(binary_predictions)
        # return np.argmax(yHat, axis=1)
    def set_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def set_activation_function(self, activation_function):
        if(activation_function == 'sigmoid'):
            self.activation = self.sigmoid
            self.activationPrime = self.sigmoidPrime
        if(activation_function == 'relu'):
            self.activation = self.relu
            self.activationPrime = self.reluPrime
        if(activation_function == 'tanh'):
            self.activation = self.tanh
            self.activationPrime = self.tanhPrime

    def set_optimizer(self, optimizer):
        self.optimizer = optimizer

    def set_hidden_layers(self, hidden_layer_sizes):
        self.hiddenLayerSize = hidden_layer_sizes
        self.num_layers = len(self.hiddenLayerSize)+1
        self.weights = []
        self.weights.append(np.random.randn(self.inputLayerSize, self.hiddenLayerSize[0]))
        for i in range(1,self.num_layers-1):
            self.weights.append(np.random.randn(self.hiddenLayerSize[i-1], self.hiddenLayerSize[i]))
        self.weights.append(np.random.randn(self.hiddenLayerSize[-1], self.outputLayerSize))
        # self.W1 = np.random.randn(self.inputLayerSize, self.hiddenLayerSize)
        # self.W2 = np.random.randn(self.hiddenLayerSize, self.outputLayerSize)
        # self.weights, self.biases = self.initialize_weights()
    def set_epochs(self, epcohs):
        self.max_iterations = epochs
        print(self.max_iterations)


In [ ]:

scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
data = pd.read_csv('advertisement.csv')
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
X = data_encoded.drop('labels', axis=1)  # Features
y = data_encoded['labels']  # Target variable
X = scaler.fit_transform(X)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y.str.split(' '))
# print(y)
y_fin = []
for i in range(len(y)):
    # print(y[i])
    # print(np.sum(y[i]))
    x = y[i]/np.sum(y[i])
    y_fin.append(x)
y = y_fin
y = np.array(y)
print(y)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
print(X_train.shape)
print(y_train.shape)

In [ ]:
mask = (y_val > 0)
y_val[mask] = 1
print(y_val)

In [ ]:
np.random.seed(0)
mlp_sgd = MultilabelMLP(inputLayerSize=1010, hiddenLayerSize=[1000], outputLayerSize=8)
mlp_sgd.set_activation_function('sigmoid')
loss = mlp_sgd.train_sgd(X_train, y_train)
y_pred = mlp_sgd.predict(X_val, y_train)
cnt = 0
for i in range(len(y_pred)):
    for j in range(len(y_pred[0])):
        if((y_pred[i][j] > 0 and y_val[i][j] == 0) or (y_pred[i][j] == 0 and y_val[i][j] > 0)):
            cnt += 1
print("Loss:", loss)
print("Accuracy:", 1-hamming_loss(y_val,y_pred))
print("F1 Score", f1_score(y_val,y_pred, average='macro'))
print("Precision", precision_score(y_val,y_pred, average='macro'))
print("Recall", recall_score(y_val,y_pred, average='macro'))
print(cnt)
print(len(y_pred)*len(y_pred[0]))

In [ ]:
print(y_train)
y_temp = copy(y_train)
mask = (y_temp > 0)
y_temp[mask] = 1
# print(y_train)
y_train_pred = mlp_sgd.predict(X_train, y_temp)
cnt = 0
for i in range(len(y_train_pred)):
    for j in range(len(y_train_pred[0])):
        if((y_train_pred[i][j] > 0 and y_temp[i][j] == 0) or (y_train_pred[i][j] == 0 and y_temp[i][j] > 0)):
            cnt += 1
print("Loss:", loss)
# print(y_train)
# print(y_train_pred)
print("Accuracy:", 1-hamming_loss(y_temp,y_train_pred))
print("F1 Score", f1_score(y_temp,y_train_pred, average='macro'))
print("Precision", precision_score(y_temp,y_train_pred, average='macro'))
print("Recall", recall_score(y_temp,y_train_pred, average='macro'))
print(cnt)
print(len(y_train_pred)*len(y_train_pred[0]))
print(y_train)

In [ ]:
activations = ["sigmoid", "tanh", "relu"]
optimizer = ["SGD","Batch","MiniBatch"]
for i in activations:
    for j in optimizer:
        np.random.seed(27)
        mlp_sgd = MultilabelMLP(inputLayerSize=1010, hiddenLayerSize=[1000], outputLayerSize=8)
        mlp_sgd.set_activation_function(i)
        mlp_sgd.set_optimizer(j)
        mlp_sgd.max_iterations = 100
        loss = mlp_sgd.train(X_train, y_train)
        y_pred = mlp_sgd.predict(X_val, y_train)
        print("Parameters: ", i, j)
        acc = 1-hamming_loss(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average='macro') 
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        print("Accuracy: ", acc, "F1 Score:", f1, "Precision:", precision, "Recall:", recall)
        

In [ ]:
# activation = ["sigmoid", "tanh", "relu"]
# optimizer = ["SGD", "Batch", "MiniBatch"]
activation = ["sigmoid"]
optimizer = ["MiniBatch"]
learning_rates = [0.1,0.01,0.001,0.0001]
epochs = [10,50,100]
hiddenlayersize = [[100,100], [100], [500], [1000], [1000,1000]]
results = []
hyperparameters = []
for i in activation:
    for j in optimizer:
        for k in learning_rates:
            for l in epochs:
                for m in hiddenlayersize:
                    mlp_sgd = MultilabelMLP(inputLayerSize=1010, hiddenLayerSize=[100], outputLayerSize=8)
                    mlp_sgd.set_activation_function(i)
                    mlp_sgd.set_optimizer(j)
                    mlp_sgd.set_learning_rate(k)
                    # print(l)
                    mlp_sgd.max_iterations = l;
                    # print(mlp_sgd.max_iterations)
                    # mlp_sgd.set_epochs(l)
                    # print(mlp_sgd.max_iterations)
                    mlp_sgd.set_hidden_layers(m)
                    # print(mlp_sgd.max_iterations)
                    loss = mlp_sgd.train(X_train, y_train)
                    y_pred = mlp_sgd.predict(X_val, y_train)
                    acc = 1-hamming_loss(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred, average='macro') 
                    precision = precision_score(y_val, y_pred, average='macro')
                    recall = recall_score(y_val, y_pred, average='macro')
                    print(acc, [i,j,k,l,m])
                    results.append([acc, f1, precision, recall])
                    hyperparameters.append([i, j, k, l, m])
result_df = pd.DataFrame(results, columns=['Accuracy', 'F1 Score', 'Precision', 'Recall'])
result_df['Activation'] = [param[0] for param in hyperparameters]
result_df['Optimizer'] = [param[1] for param in hyperparameters]
result_df['Learning Rate'] = [param[2] for param in hyperparameters]
result_df['Epochs'] = [param[3] for param in hyperparameters]
result_df['Hidden Layer Size'] = [param[4] for param in hyperparameters]

# Find the hyperparameter combination with the highest accuracy
best_hyperparameters = result_df.loc[result_df['Accuracy'].idxmax()]

# Print the best hyperparameter combination and its corresponding metrics
print("Best Hyperparameters:")
print(best_hyperparameters)

# Print the results table
print("\nResults Table:")
print(result_df)            
                    


In [ ]:
np.random.seed(0)
mlp_sgd = MultilabelMLP(inputLayerSize=1010, hiddenLayerSize=[1000], outputLayerSize=8)
mlp_sgd.set_activation_function('sigmoid')
loss = mlp_sgd.train_sgd(X_train, y_train)
y_pred = mlp_sgd.predict(X_test, y_train)
mask = (y_test > 0)
y_test[mask] = 1
print(y_test)
cnt = 0
for i in range(len(y_pred)):
    for j in range(len(y_pred[0])):
        if((y_pred[i][j] > 0 and y_test[i][j] == 0) or (y_pred[i][j] == 0 and y_test[i][j] > 0)):
            cnt += 1
print("Loss:", loss)
print("Accuracy:", 1-hamming_loss(y_test,y_pred))
print("F1 Score", f1_score(y_test,y_pred, average='macro'))
print("Precision", precision_score(y_test,y_pred, average='macro'))
print("Recall", recall_score(y_test,y_pred, average='macro'))
print(cnt)
print(len(y_pred)*len(y_pred[0]))